Initialization 

Just loading packages and connecting to database. 

In [1]:
from telnetlib import STATUS
import psycopg2
import pandas as pd
import sys
from tqdm import tqdm
sys.path.append('H:/uth_helpers')
from db_utils import get_dsn

connection = psycopg2.connect(get_dsn())
connection.autocommit = True


C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Row Counts And Enrollment Counts

Getting total row counts, member ID counts, and claim counts for the data warehouse and the raw data. 

In [13]:
tableCreateQuery = '''
    DROP TABLE IF EXISTS qa_reporting.dw_iqva_claim_header_counts; 

    CREATE TABLE qa_reporting.dw_iqva_claim_header_counts (
        calender_year smallint, 
        table_src TEXT, 
        dw_row_count INT, 
        src_row_count INT, 
        row_count_diff INT, 
        row_count_diff_percentage FLOAT, 
        dw_uth_clm_id_count INT, 
        dw_src_clm_id_count INT, 
        src_clm_count INT, 
        clm_count_diff INT, 
        clm_count_percentage FLOAT, 
        dw_uth_mbr_id_count INT, 
        dw_src_mbr_id_count INT, 
        src_mbr_count INT, 
        mbr_count_diff INT, 
        mbr_count_percentage FLOAT, 
        date_generated DATE
    ); 
'''

with connection.cursor() as cursor: 
    cursor.execute(tableCreateQuery)

In [14]:
with connection.cursor() as cursor: 
    countInsertQuery = '''
        INSERT INTO qa_reporting.dw_iqva_claim_header_counts
        (
            calender_year, 
            table_src, 
            dw_row_count, 
            date_generated
        )
        SELECT 
            year,
            'claims' as table_id_src, 
            COUNT(*), 
            current_date
        FROM dw_staging.iqva_claim_header
        GROUP BY 1; 
    '''

    cursor.execute(countInsertQuery)


In [ ]:
with connection.cursor() as cursor:

    mbrCountUpdateQuery = f'''
        UPDATE qa_reporting.dw_iqva_claim_detail_counts b
        SET dw_uth_mbr_id_count = count
        FROM (
            SELECT 
                year, 
                COUNT(DISTINCT uth_member_id) AS count 
            FROM dw_staging.iqva_claim_header
            GROUP BY 1
        ) a
        WHERE a.year = b.calendar_year
    '''

    cursor.execute(mbrCountUpdateQuery)

In [ ]:
with connection.cursor() as cursor:

    srcCountUpdateQuery = f'''
        UPDATE qa_reporting.dw_iqva_claim_detail_counts b
        SET dw_src_clm_id_count = clm_count, dw_src_mbr_id_count = mbr_count
        FROM (
            SELECT 
                year, 
                COUNT(DISTINCT claim_id_src) AS clm_count, 
                COUNT(DISTINCT member_id_src) AS mbr_count 
            FROM dw_staging.iqva_claim_header
            GROUP BY 1
        ) a
        WHERE a.year = b.calendar_year
    '''

    cursor.execute(srcCountUpdateQuery)

In [15]:
with connection.cursor() as cursor: 

    clmCountUpdateQuery = '''
        UPDATE qa_reporting.dw_iqva_claim_header_counts b 
        SET dw_uth_clm_id_count = count 
        FROM (
            SELECT 
                year, 
                COUNT(DISTINCT uth_claim_id) AS count
            FROM dw_staging.iqva_claim_header
            GROUP BY 1
        ) a
        WHERE a.year = b.calender_year; 
    '''

    cursor.execute(clmCountUpdateQuery)

In [32]:
with connection.cursor() as cursor: 

    compUpdateQuery = '''
        WITH clms AS (
            SELECT 
                min(year) as year, 
                pat_id, 
                derv_claimno
            FROM dev.pi_iqvia_derv_claimno_new_all_yr
            WHERE new_rectype != 'P' OR new_rectype IS NULL
            GROUP BY pat_id, derv_claimno
        ),
        clm_counts AS (
            SELECT 
                year, 
                COUNT(*) row_count, 
                COUNT(DISTINCT pat_id) pat_count, 
                COUNT(DISTINCT derv_claimno) clm_count
            FROM clms
            GROUP BY 1
        )
        UPDATE qa_reporting.dw_iqva_claim_header_counts a 
        SET 
            src_row_count = b.row_count, 
            row_count_diff = dw_row_count - b.row_count,
            row_count_diff_percentage = 100. * abs( dw_row_count - b.row_count) / b.row_count,
            src_clm_count = clm_count,
            clm_count_diff = dw_uth_clm_id_count - b.clm_count,
            clm_count_percentage = 100. * abs(dw_uth_clm_id_count - b.clm_count) / b.clm_count,
            src_mbr_count = b.pat_count,
            mbr_count_diff = dw_uth_mbr_id_count - b.pat_count,
            mbr_count_percentage = 100. * abs(dw_uth_mbr_id_count - b.pat_count) / b.pat_count
        FROM clm_counts b 
        WHERE a.calender_year = b.year; 
    '''

    cursor.execute(compUpdateQuery)

In [33]:
countsDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_claim_header_counts;', con=connection)
countsDf.sort_values('calender_year')
print(countsDf)

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_45624\3197021241.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  countsDf = pd.read_sql('SELECT * FROM qa_reporting.dw_iqva_claim_header_counts;', con=connection)


    calender_year table_src  dw_row_count  src_row_count  row_count_diff  \
0            2018    claims     145819826      145819826               0   
1            2020    claims     117291638      117291638               0   
2            2008    claims     354729976      354729976               0   
3            2021    claims     130451913      130451913               0   
4            2014    claims     258599579      258599579               0   
5            2009    claims     356427314      356427314               0   
6            2019    claims     147254547      147254547               0   
7            2016    claims     256236196      256236196               0   
8            2023    claims      45991710       45991710               0   
9            2015    claims     244283420      244283420               0   
10           2007    claims     318812171      318812171               0   
11           2010    claims     311027752      311027752               0   
12          

In [31]:
countsDf['row_count_diff_percentage'].describe()

count    18.000000
mean      0.006755
std       0.003948
min       0.000000
25%       0.004691
50%       0.005661
75%       0.007474
max       0.016933
Name: row_count_diff_percentage, dtype: float64

In [33]:
countsDf['clm_count_percentage'].describe()

count    18.000000
mean      0.006755
std       0.003948
min       0.000000
25%       0.004691
50%       0.005661
75%       0.007474
max       0.016933
Name: clm_count_percentage, dtype: float64

In [34]:
countsDf['mbr_count_percentage'].describe()

count    18.000000
mean      0.002325
std       0.003034
min       0.000000
25%       0.000972
50%       0.001272
75%       0.001924
max       0.012537
Name: mbr_count_percentage, dtype: float64

Claim Type

Now that row counts, member ID counts, and claim ID counts have been verified as accurate, we can get counts for different claim types. 

In [25]:
with connection.cursor() as cursor:
    clmTypeQuery = '''
        DROP TABLE IF EXISTS qa_reporting.iqva_claim_type_count;
        with cleaned_billtypes as (
            SELECT
                year,
                new_rectype,
                pat_id, 
                derv_claimno,
                CASE 
                    WHEN LENGTH(TRIM(billtype)) = 1 THEN NULL 
                    WHEN SUBSTRING(billtype, 1, 1) NOT IN ('1', '2', '3', '4', '5', '6', '7', '8', '9') THEN NULL 
                    WHEN SUBSTRING(billtype, 2, 1) NOT IN ('1', '2', '3',' 4', '5', '6', '7', '8', '9') THEN NULL 
                    WHEN length(trim(billtype)) = 2 THEN rpad(billtype, 3, '1') 
                    WHEN SUBSTRING(billtype, 3, 1) = '?' THEN replace(billtype, '?', '1') 
                    WHEN SUBSTRING(billtype, 3, 1) NOT IN ('0', '1', '2', '3', '4', '5', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'M', 'O', 'P', 'Q', 'X', 'Y', 'Z') THEN NULL 
                    ELSE billtype 
                END AS billtype_cleaned, 
                CASE 
                    WHEN rev_code in ('000', '0000') THEN NULL 
                    WHEN length(trim(rev_code)) in (1, 2) THEN NULL 
                    WHEN SUBSTRING(rev_code, 1, 1) NOT IN ('0', '1', '2', '3') THEN NULL 
                    WHEN SUBSTRING(rev_code, 2, 1) NOT IN ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9') THEN NULL 
                    WHEN SUBSTRING(rev_code, 3, 1) NOT IN ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9') THEN NULL 
                    WHEN length(trim(rev_code)) = '3' THEN lpad(rev_code, 4, '0')
                    WHEN SUBSTRING(rev_code, 4, 1) NOT IN ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9') THEN NULL 
                    ELSE rev_code 
                END AS rev_code_cleaned
            FROM dev.pi_iqvia_derv_claimno_new_all_yr
        ),
        claim_types AS (
            SELECT
                a.year as year,
                a.rev_code_cleaned,
                a.pat_id, 
                a.derv_claimno,
                CASE 
                    WHEN (a.billtype_cleaned is not NULL or b.revenue_cd is not NULL) or (a.billtype_cleaned != '' or b.revenue_cd != '') THEN 'F' 
                    WHEN ((a.billtype_cleaned is NULL and b.revenue_cd is NULL) or (a.billtype_cleaned = '' and b.revenue_cd = '')) and a.new_rectype in ('F', 'S') THEN 'F' 
                    ELSE 'P'
                END AS claim_type
            FROM cleaned_billtypes a
            LEFT JOIN reference_tables.ref_revenue_code b ON a.rev_code_cleaned = b.revenue_cd
        ), 
        grouped_claim_types AS (
            SELECT 
                min(year) AS year, 
                min(claim_type) AS claim_type, 
                pat_id, 
                derv_claimno
            FROM claim_types
            GROUP BY pat_id, derv_claimno
        )
        SELECT 
            year, 
            claim_type, 
            COUNT(*)
        INTO qa_reporting.iqva_claim_type_count
        FROM grouped_claim_types
        GROUP BY 1,2
    '''
    cursor.execute(clmTypeQuery)

In [76]:
with connection.cursor() as cursor:
    query = '''
        DROP TABLE IF EXISTS qa_reporting.dw_iqva_claim_type_count;
        SELECT 
            year, 
            claim_type, 
            COUNT(*)
        INTO qa_reporting.dw_iqva_claim_type_count
        FROM dw_staging.iqva_claim_header
        GROUP BY 1,2
    '''
    cursor.execute(query)

In [27]:
query = f'''
        SELECT 
                a.year, 
                a.claim_type, 
                a.count AS dw_claim_type_count, 
                b.count AS src_claim_type_count, 
                a.count-b.count AS claim_type_count_difference, 
                100. * abs(a.count-b.count) / b.count AS claim_type_count_difference_percentage
        FROM qa_reporting.dw_iqva_claim_type_count a
        FULL OUTER JOIN qa_reporting.iqva_claim_type_count b ON a.year = b.year AND a.claim_type = b.claim_type
        WHERE a.claim_type != 'P' AND b.claim_type != 'P'
'''

pd.read_sql(query, con=connection).sort_values(['year', 'claim_type'])

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_45624\4144069886.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, con=connection).sort_values(['year', 'claim_type'])


,year,claim_type,dw_claim_type_count,src_claim_type_count,claim_type_count_difference,claim_type_count_difference_percentage
12,2006,F,40728907,40728907,0,0.000000
15,2007,F,51221215,51221215,0,0.000000
9,2008,F,62502955,62502963,-8,0.000013
3,2009,F,63667177,63667178,-1,0.000002
14,2010,F,58220378,58220378,0,0.000000
4,2011,F,58836551,58836552,-1,0.000002
8,2012,F,54615979,54615979,0,0.000000
1,2013,F,52856925,52856925,0,0.000000
5,2014,F,56322002,56322003,-1,0.000002
10,2015,F,52546408,52546408,0,0.000000
